In [5]:
import torch
from torch import nn

In [11]:
x = torch.arange(4.)
k = x.clone()

x = x.reshape(1,1,2,2)
k = k.reshape(1,1,2,2)

tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, bias=False)
tconv.weight.data = k
tconv(x)

tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, stride=2, bias=False)
tconv.weight.data = k
tconv(x)

tensor([[[[0., 0., 0., 1.],
          [0., 0., 2., 3.],
          [0., 2., 0., 3.],
          [4., 6., 6., 9.]]]], grad_fn=<ConvolutionBackward0>)